Dataset : https://drive.google.com/drive/folders/1Eb8VgRaabdD00AdFDnAOi3xmG4uzHxdO?usp=drive_link

### Testing and Prediction

In [ ]:
import os
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# Define the model class
class RES_EfficientCNN(nn.Module):
    def __init__(self, num_classes=2):
        super(RES_EfficientCNN, self).__init__()
        # Load a pre-trained EfficientNet model
        self.base_model = models.efficientnet_b0(pretrained=True)  # Use the latest weights
        # Replace the classifier with a new one for your specific task
        in_features = self.base_model.classifier[1].in_features
        self.base_model.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.base_model(x)

# Initialize the model
model = RES_EfficientCNN(num_classes=2)

# Load the trained model
model_path = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/Custom_EfficientNet_B0.pth'  # Adjust the path to Custom_EfficientNet_B0 model
model.load_state_dict(torch.load(model_path))
model.eval()  # Set the model to evaluation mode

# Define the image transformation (adjust as per your training process)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # Normalize for 3 channels
])

# Source directory where the generated spectrogram images are stored
test_spectrogram_dir = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/selected_audio_files_testing_spectrograms'

# Process each spectrogram image and make predictions
predictions = []
for file_name in os.listdir(test_spectrogram_dir):
    if file_name.endswith('.png'):  # Ensure we're working with image files
        image_path = os.path.join(test_spectrogram_dir, file_name)
        image = Image.open(image_path).convert('L')  # Convert to grayscale
        image = image.convert('RGB')  # Convert to RGB to match model input
        image = transform(image).unsqueeze(0)  # Add batch dimension

        # Make prediction
        with torch.no_grad():  # Disable gradient calculation for inference
            output = model(image)
            _, predicted = torch.max(output, 1)  # Get the class with the highest score

        predictions.append((file_name, predicted.item()))  # Store filename and prediction

# Optionally, save or print the predictions
for filename, prediction in predictions:
    print(f"{filename}: {prediction}")

# If you want to save the predictions to a file
output_file = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/predictions.txt'
with open(output_file, 'w') as f:
    for filename, prediction in predictions:
        f.write(f"{filename}: {prediction}\n")

print("Predictions have been successfully generated.")


<ipython-input-6-deefc85bf76f>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


LA_E_2602463.png: 1
LA_E_8974701.png: 1
LA_E_5224411.png: 1
LA_E_9629099.png: 1
LA_E_1572528.png: 1
LA_E_4505498.png: 1
LA_E_8478765.png: 1
LA_E_3651348.png: 1
LA_E_1147384.png: 1
LA_E_8023690.png: 1
LA_E_5835667.png: 1
LA_E_5762961.png: 1
LA_E_5978185.png: 1
LA_E_6479378.png: 1
LA_E_8579243.png: 1
LA_E_6210128.png: 0
LA_E_4924002.png: 0
LA_E_4793366.png: 1
LA_E_1411561.png: 0
LA_E_4455219.png: 1
LA_E_8996199.png: 0
LA_E_2001230.png: 1
LA_E_1671939.png: 1
LA_E_4271320.png: 1
LA_E_1890993.png: 1
LA_E_5009251.png: 0
LA_E_1830307.png: 1
LA_E_5624940.png: 1
LA_E_4306145.png: 0
LA_E_2407430.png: 1
LA_E_2218533.png: 1
LA_E_6927350.png: 1
LA_E_6273262.png: 1
LA_E_7850546.png: 1
LA_E_6782590.png: 1
LA_E_3358578.png: 1
LA_E_8535405.png: 1
LA_E_3203713.png: 1
LA_E_8445983.png: 1
LA_E_5742134.png: 1
LA_E_7172071.png: 1
LA_E_2733438.png: 0
LA_E_9928530.png: 1
LA_E_6394190.png: 1
LA_E_9055968.png: 1
LA_E_5367430.png: 1
LA_E_4116166.png: 1
LA_E_4042297.png: 1
LA_E_9872320.png: 1
LA_E_1468379.png: 1


## Convert txt to csv

In [ ]:
import pandas as pd
import os

# Path to the predicted labels txt file
predicted_labels_txt = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/predictions.txt'

# Load the predicted labels into a DataFrame with column names matching the original labels file
predicted_labels_df = pd.read_csv(predicted_labels_txt, delim_whitespace=True, header=None, names=['FileName', 'SpoofType'])

# Remove file extensions from 'FileName' column
predicted_labels_df['FileName'] = predicted_labels_df['FileName'].apply(lambda x: os.path.splitext(x)[0])

# Path to save the predicted labels CSV file
predicted_labels_csv = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/predictions.csv'

# Save the DataFrame as a CSV file
predicted_labels_df.to_csv(predicted_labels_csv, index=False)

print(f"Predicted labels saved to {predicted_labels_csv}")




Predicted labels saved to /content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/predictions.csv


<ipython-input-5-6e3939b9b596>:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  predicted_labels_df = pd.read_csv(predicted_labels_txt, delim_whitespace=True, header=None, names=['FileName', 'SpoofType'])


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score

# Load the original and predicted label files
original_labels_csv = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/selected_audio_files_testing_labels.csv'
predicted_labels_csv = '/content/drive/MyDrive/ADD_ASV_DATA/LA/ASVspoof2019_LA_train/ADD_CNN/Testing_RES_On_Unknown_Data/predictions.csv'

# Read the original and predicted CSV files into DataFrames
original_df = pd.read_csv(original_labels_csv)
predicted_df = pd.read_csv(predicted_labels_csv)

# Map the numeric predictions to their string equivalents
predicted_df['SpoofType'] = predicted_df['SpoofType'].map({0: 'bonafide', 1: 'spoof'})

# Merge the two DataFrames on the 'FileName' column to align the labels
merged_df = pd.merge(original_df, predicted_df, on='FileName', suffixes=('_original', '_predicted'))

# Calculate accuracy by comparing the 'SpoofType' columns
accuracy = accuracy_score(merged_df['SpoofType_original'], merged_df['SpoofType_predicted'])

# Print the accuracy
print(f"Testing Accuracy: {accuracy * 100:.2f}%")


Testing Accuracy: 80.55%
